In [1]:
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate

# For caching models and logging
!pip install huggingface_hub

In [2]:
import torch

# Check GPU status
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Available Memory: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")

# Clear cache
torch.cuda.empty_cache()

GPU: Tesla T4
Total Memory: 15.83 GB
Available Memory: 15.83 GB


In [ ]:
import os
from huggingface_hub import login

# Load token from environment variable
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

In [4]:
import torch
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())


PyTorch: 2.8.0+cu128
CUDA available: True
CUDA version: 12.8
cuDNN version: 91002


In [6]:
import os
os.environ['VLLM_USE_V1'] = '0'

import torch
import gc

torch.cuda.empty_cache()
gc.collect()

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        with torch.cuda.device(i):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()

import pandas as pd
from vllm import LLM, SamplingParams
from tqdm import tqdm

df = pd.read_csv('/kaggle/input/banglachq/train.csv')
df = df[['id', 'question']].copy()
print(f"Loaded {len(df)} rows")

model_name = "google/gemma-3-1b-it"
print(f"\nLoading {model_name}...")

llm = LLM(
    model=model_name,
    trust_remote_code=True,
    dtype="float32",  # 1B should work with float16 on T4
    gpu_memory_utilization=0.7,
    max_model_len=2048,
    max_num_seqs=24,
)

tokenizer = llm.get_tokenizer()
print("Model loaded!\n")

# keep prompt intact
def translate_to_sylheti(bangla_text):
    prompt = f"""You are a precise translation tool. Your task is to translate the given Bangla text to Sylheti dialect using Bengali script.

        INSTRUCTIONS:
        - Analyze the text step-by-step before translating
        - Use only Bengali script in your final translation
        - After your reasoning, provide ONLY the final translation with no additional commentary

        Here are examples showing the translation process:

        Example 1:
        Bangla: আপনি কি নিয়মিত স্ক্যান করেন?
        Reasoning: "আপনি" becomes "আফনে" (phonetic shift প→ফ, ই→ে), "কি নিয়মিত" becomes "কিতা রেগুলার" (কি→কিতা with emphasis particle, নিয়মিত→রেগুলার lexical change), "স্ক্যান করেন" becomes "স্ক্যান খরইননি?" (করেন→খরইননি with খ sound and -ননি ending for question)
        Sylheti: আফনে কিতা রেগুলার স্ক্যান খরইননি?

        Example 2:
        Bangla: কোথায় কোচিং করেন?
        Reasoning: "কোথায়" becomes "কোনানো" (locative form কোথায়→কোনানো with extended vowel), "কোচিং" stays as "কোচিং" (unchanged), "করেন" becomes "খরইন?" (verb করেন→খরইন with খ sound and -ইন ending)
        Sylheti: কোনানো কোচিং খরইন?

        Example 3:
        Bangla: ডাল সিদ্ধ হয়েছে?
        Reasoning: "ডাল" becomes "ডাইল" (vowel shift আ→আই for diphthong), "সিদ্ধ" stays as "সিদ্ধ" (unchanged), "হয়েছে" becomes "অইছেনি?" (হয়েছে→অইছেনি with অ prefix and -নি question marker)
        Sylheti: ডাইল সিদ্ধ অইছেনি?

        Example 4:
        Bangla: এই যে মিয়া, কি নিয়ে যাও?
        Reasoning: "এই যে মিয়া" becomes "ওবা" (colloquial address form, complete lexical replacement for calling attention), "কি নিয়ে" becomes "কিতা লইয়া" (কি→কিতা, নিয়ে→লইয়া with ল sound), "যাও" becomes "যাইরায়?" (যাও→যাইরায় with extended verb form and য় ending)
        Sylheti: ওবা, কিতা লইয়া যাইরায়?

        Example 5:
        Bangla: জুম্মার পর যাব।
        Reasoning: "জুম্মার" becomes "জুম্মার" (unchanged as Arabic loanword), "পর" stays as "পর" (unchanged), "যাব" becomes "যামু" (first person future -ব→-মু typical Sylheti verb ending)
        Sylheti: জুম্মার পর যামু।

        Now translate this text:
        Bangla: {bangla_text}

        Think through the translation:
        - Identify key vocabulary transformations
        - Note grammatical changes (verb forms, pronouns, question markers)
        - Apply Sylheti phonetic patterns (খ for ক in verbs, diphthongs, vowel shifts)

        Provide your final translation below:
        Sylheti:""" 
    
    messages = [{"role": "user", "content": prompt}]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

sampling_params = SamplingParams(
    temperature=0.3,
    top_p=0.9,
    max_tokens=512,
    stop=["Bangla:"]
)

BATCH_SIZE = 24
all_translations = []

for batch_start in tqdm(range(0, len(df), BATCH_SIZE), desc="Translating"):
    batch_end = min(batch_start + BATCH_SIZE, len(df))
    batch_prompts = [translate_to_sylheti(row['question']) for _, row in df.iloc[batch_start:batch_end].iterrows()]
    
    outputs = llm.generate(batch_prompts, sampling_params)
    all_translations.extend([o.outputs[0].text.strip() for o in outputs])
    
    del outputs, batch_prompts
    gc.collect()
    torch.cuda.empty_cache()

df['sylheti_translation'] = all_translations
df.to_csv('gemma_5_shot_sylheti_CoT.csv', index=False)
print("Done!")


2025-09-28 17:38:01.028999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759081081.054519     488 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759081081.062854     488 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 09-28 17:38:06 [__init__.py:216] Automatically detected platform cuda.
Loaded 1880 rows

Loading google/gemma-3-1b-it...
INFO 09-28 17:38:07 [utils.py:328] non-default args: {'trust_remote_code': True, 'dtype': 'float32', 'max_model_len': 2048, 'gpu_memory_utilization': 0.7, 'max_num_seqs': 24, 'disable_log_stats': True, 'model': 'google/gemma-3-1b-it'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 09-28 17:38:26 [__init__.py:742] Resolved architecture: Gemma3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


...

Translating:  99%|█████████▊| 78/79 [27:26<00:22, 22.00s/it]

Adding requests:   0%|          | 0/8 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/8 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Translating: 100%|██████████| 79/79 [27:41<00:00, 21.03s/it]

Done!
